# Step 1 | Platform Setup¶

## Step 1.1 | Check Environment

1. Open Anaconda Prompt
2. conda activate tf-gpu
3. cd "C:\Users\FaithanTo\Desktop\MSBA 6421 (001) Predictive Analytics\m5-forecasting-accuracy"
4. jupyter notebook


In [2]:
!where python

C:\Anaconda\envs\tf-gpu\python.exe
C:\Anaconda\python.exe


In [3]:
import sys
import tensorflow as tf
import torch

print(sys.executable)
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

C:\Anaconda\envs\tf-gpu\python.exe
2.10.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1.13.1+cu116
True
NVIDIA GeForce RTX 3050 4GB Laptop GPU


## Step 1.2 | Import Libraries

In [4]:
import polars as pl
import pandas as pd
import numpy as np
import random
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import gc
import warnings
import os
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm
from multiprocess import Pool, cpu_count
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing
import pickle
import joblib
import glob
import psutil
import os
from m5_wrmsse import wrmsse
import csv

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = ""
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
pd.set_option('display.max_columns', None)
print("NumPy version:", np.__version__)

NumPy version: 1.26.4


# Step 2 | Load Data

In [6]:
# Load data as a Polars DataFrame

# This tells Polars to cache categorical strings globally, so the encodings remain consistent across multiple DataFrames 
# - essential for clean joins or filters later in your pipeline.
pl.enable_string_cache()

df_pl = pl.read_parquet("C:/Users/FaithanTo/Desktop/MSBA 6421 (001) Predictive Analytics/m5-forecasting-accuracy/Step3_Encoding_LSTM_v3.parquet")
print(f"✅ Loaded dataframe with shape: {df_pl.shape}")
df_pl.head()

✅ Loaded dataframe with shape: (59181090, 32)


id,item_id,dept_id,cat_id,store_id,state_id,sales,year,event_name_1,event_type_1,snap_CA,snap_TX,snap_WI,sell_price,d_num,store_item_rolling_mean_7d,store_item_rolling_std_30d,store_total_sales_7d,store_avg_item_sales_ratio,store_total_sales,store_cat_rolling_mean_14d,state_dept_rolling_mean_30d,state_cat_prev_week_sales,state_cat_weekly_growth,country_sales,sales_LY,is_new_id,d_num_scaled,weekday_sin,weekday_cos,month_sin,month_cos
cat,i32,i32,i32,i32,i32,i16,i64,i32,i32,i8,i8,i8,f64,i16,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i8,f64,f64,f64,f64,f64
"""FOODS_1_001_CA_1_evaluation""",0,0,0,0,0,3,0,19,2,0,0,0,-0.693997,1,0.551939,0.574769,-2.431641,0.304447,0.674762,1.696726,-0.545545,1.464391,-0.031893,-0.249161,0.458554,1,0.0,-0.974928,-0.222521,0.0,1.0
"""FOODS_1_001_CA_1_evaluation""",0,0,0,0,0,0,0,19,2,0,0,0,-0.693997,2,0.10919,0.574769,-1.920052,-0.273864,0.53661,1.648795,-0.548824,1.464391,-0.031893,-0.367965,0.458554,1,0.000515,-0.781831,0.62349,0.0,1.0
"""FOODS_1_001_CA_1_evaluation""",0,0,0,0,0,0,0,19,2,0,0,0,-0.693997,3,-0.038393,0.370301,-1.573328,-0.273864,-0.479796,1.279138,-0.601656,1.464391,-0.031893,-1.440972,0.458554,1,0.001031,0.0,1.0,0.0,1.0
"""FOODS_1_001_CA_1_evaluation""",0,0,0,0,0,1,0,19,2,1,1,0,-0.693997,4,-0.038393,0.203353,-1.19767,0.000159,-0.301412,1.153047,-0.632881,1.464391,-0.031893,-1.221549,0.458554,1,0.001546,0.781831,0.62349,0.5,0.866025
"""FOODS_1_001_CA_1_evaluation""",0,0,0,0,0,4,0,19,2,1,0,1,-0.693997,5,0.138707,0.414706,-0.873848,0.997689,-0.620984,1.034914,-0.652841,1.464391,-0.031893,-2.065569,0.458554,1,0.002062,0.974928,-0.222521,0.5,0.866025


# Step 3 | Per-ID Time Series Partitioning for Memory-Safe Sequencing

In [17]:
# === Directory ===
SAVE_DIR = "sequence_chunks_v3"
GROUP_SAVE_DIR = os.path.join(SAVE_DIR, "grouped_v3")
os.makedirs(GROUP_SAVE_DIR, exist_ok=True)

## Step 3.1 | Create Partitions

In [8]:
# === Get unique IDs ===
all_ids = df_pl.select("id").unique().to_series().to_list()
print(f"🧾 Found {len(all_ids):,} unique IDs")

🧾 Found 30,490 unique IDs


In [9]:
# === Helper: Show memory usage ===
def print_memory(label=""):
    mem = psutil.Process().memory_info().rss / 1024**3
    print(f"🧠 Memory [{label}]: {mem:.2f} GB")

In [10]:
# === Save each group (resume-safe) ===
print("📦 Saving individual ID-level parquet files to disk...")
print_memory("Start")

for i, id_val in enumerate(tqdm(all_ids)):
    save_path = os.path.join(GROUP_SAVE_DIR, f"{id_val}.parquet")

    if not os.path.exists(save_path):
        df_pl.filter(pl.col("id") == id_val).write_parquet(save_path)

    # Free memory and monitor usage every 1000 IDs
    if i % 1000 == 0 and i > 0:
        gc.collect()
        print_memory(f"After {i} IDs")

# Final cleanup
gc.collect()
print_memory("Final")

print("✅ Done saving all group files.")

📦 Saving individual ID-level parquet files to disk...
🧠 Memory [Start]: 2.91 GB


  3%|██▍                                                                        | 1003/30490 [01:38<1:48:50,  4.51it/s]

🧠 Memory [After 1000 IDs]: 3.26 GB


  7%|████▉                                                                      | 2002/30490 [03:21<1:04:03,  7.41it/s]

🧠 Memory [After 2000 IDs]: 3.61 GB


...

🧠 Memory [After 29000 IDs]: 8.55 GB


 98%|██████████████████████████████████████████████████████████████████████████▊ | 30003/30490 [57:17<01:04,  7.57it/s]

🧠 Memory [After 30000 IDs]: 8.76 GB


100%|████████████████████████████████████████████████████████████████████████████| 30490/30490 [58:12<00:00,  8.73it/s]


🧠 Memory [Final]: 8.01 GB
✅ Done saving all group files.


## Step 3.2 | Verify Partitions

### Step 3.2.1 | File Count Check

In [9]:
expected_ids = df_pl.select("id").unique().to_series().to_list()

# Check number of saved files
saved_files = [f for f in os.listdir(GROUP_SAVE_DIR) if f.endswith(".parquet")]

print(f"Expected ID count: {len(expected_ids):,}")
print(f"Saved parquet files: {len(saved_files):,}")

missing_files = set(expected_ids) - set(f.replace(".parquet", "") for f in saved_files)
if missing_files:
    print(f"❌ {len(missing_files)} files are missing!")
else:
    print("✅ All IDs are accounted for.")

Expected ID count: 30,490
Saved parquet files: 30,490
✅ All IDs are accounted for.


### Step 3.2.2 | Spot Check Contents

In [10]:
sample_files = random.sample(saved_files, 5)

for f in sample_files:
    df = pl.read_parquet(os.path.join(GROUP_SAVE_DIR, f))
    unique_ids = df.select("id").unique().to_series().to_list()
    dnum_sorted = df["d_num"].is_sorted()

    print(f"🔍 {f}: {len(df)} rows, ID = {unique_ids[0]}")
    if len(unique_ids) != 1:
        print("❌ Multiple IDs found!")
    if not dnum_sorted:
        print("⚠️ d_num is not sorted!")

🔍 HOBBIES_1_104_CA_4_evaluation.parquet: 1941 rows, ID = HOBBIES_1_104_CA_4_evaluation
🔍 FOODS_3_519_CA_2_evaluation.parquet: 1941 rows, ID = FOODS_3_519_CA_2_evaluation
🔍 HOBBIES_1_307_CA_1_evaluation.parquet: 1941 rows, ID = HOBBIES_1_307_CA_1_evaluation
🔍 FOODS_3_232_CA_2_evaluation.parquet: 1941 rows, ID = FOODS_3_232_CA_2_evaluation
🔍 FOODS_3_585_TX_1_evaluation.parquet: 1941 rows, ID = FOODS_3_585_TX_1_evaluation


### Step 3.2.3 | Check Column Names Match

In [13]:
expected_cols = df_pl.columns
bad_files = []

for f in random.sample(saved_files, 20):
    df = pl.read_parquet(os.path.join(GROUP_SAVE_DIR, f))
    if df.columns != expected_cols:
        print(f"❌ Column mismatch in {f}")
        bad_files.append(f)

bad_files

[]

### Step 3.2.4 | Recreate ID Summary Table

In [14]:
# Recreate a small summary like: ID, num_rows, d_min, d_max
summary_rows = []

for f in random.sample(saved_files, 10):
    df = pl.read_parquet(os.path.join(GROUP_SAVE_DIR, f))
    summary_rows.append({
        "id": df["id"][0],
        "num_rows": df.shape[0],
        "min_d": df["d_num"].min(),
        "max_d": df["d_num"].max()
    })

pd.DataFrame(summary_rows)

,id,num_rows,min_d,max_d
0,FOODS_3_509_TX_1_evaluation,1941,1,1941
1,FOODS_3_459_TX_3_evaluation,1941,1,1941
2,HOUSEHOLD_1_231_CA_4_evaluation,1941,1,1941
3,HOUSEHOLD_1_195_CA_3_evaluation,1941,1,1941
4,FOODS_2_022_WI_3_evaluation,1941,1,1941
5,FOODS_3_420_WI_2_evaluation,1941,1,1941
6,HOBBIES_2_084_CA_1_evaluation,1941,1,1941
7,FOODS_3_136_WI_1_evaluation,1941,1,1941
8,FOODS_1_084_CA_4_evaluation,1941,1,1941
9,HOBBIES_1_362_CA_4_evaluation,1941,1,1941


# Step 4 | Sequencing

## Step 4.1 | Input Columns

In [12]:
embedding_cols = ['item_id', 'dept_id']

categorical_cols = [
    'cat_id',
    'store_id', 'state_id',
    'year', 
    'event_name_1', 'event_type_1'
]

cycle_cols = [
    'weekday_sin', 'weekday_cos',
    'month_sin', 'month_cos',
]

binary_cols = [
    'snap_CA', 'snap_TX', 'snap_WI', 
    'is_new_id'
]

numerical_cols_ss = [
    'sell_price', 'sales_LY',
    'store_item_rolling_mean_7d', 'store_item_rolling_std_30d',
    'store_total_sales_7d', 'store_avg_item_sales_ratio',
    'store_total_sales', 'store_cat_rolling_mean_14d',
    'state_dept_rolling_mean_30d', 'state_cat_prev_week_sales',
    'state_cat_weekly_growth', 'country_sales'
]

numerical_cols_mm = ['d_num_scaled']

input_cols = embedding_cols + categorical_cols + cycle_cols + binary_cols + numerical_cols_ss + numerical_cols_mm

In [13]:
# === Parameters ===
INPUT_SEQ_LEN = 56
FORECAST_HORIZON = 28
STRIDE = 7
BATCH_SIZE = 100
SEED = 42

## Step 4.2 | Create Training Sequences

In [26]:
# === Sequence Generator ===
def generate_sequences_for_id_polars(group_df, input_cols, input_len, output_len, stride=1):
    X, y = [], []
    group_df = group_df.sort("d_num")
    df_len = group_df.height

    if df_len < input_len + output_len:
        return X, y

    input_array = group_df.select(input_cols).to_numpy()
    sales_array = group_df.select("sales").to_numpy().flatten()

    for i in range(0, df_len - input_len - output_len + 1, stride):
        target_window = group_df[i + input_len : i + input_len + output_len]

        # ✅ Safely check the final d_num of the target window
        if target_window[-1, "d_num"] < 1914:
            X.append(input_array[i:i+input_len].astype(np.float32))
            y.append(sales_array[i+input_len:i+input_len+output_len].astype(np.float32))

    return X, y

In [27]:
# === Randomized Batching ===
def get_randomized_batches(group_dir, batch_size, seed=42):
    all_ids = [f.replace(".parquet", "") for f in os.listdir(group_dir) if f.endswith(".parquet")]
    random.seed(seed)
    random.shuffle(all_ids)
    return [all_ids[i:i+batch_size] for i in range(0, len(all_ids), batch_size)]

In [28]:
# === Dispatcher ===
def serial_generate_from_disk(group_dir, save_dir, input_cols, input_len, output_len, stride, batch_size):
    os.makedirs(save_dir, exist_ok=True)
    batches = get_randomized_batches(group_dir, batch_size)

    start = time.time()
    for idx, batch in enumerate(tqdm(batches, desc="Saving shuffled batches")):
        X_all, y_all = [], []
        for id_val in batch:
            path = os.path.join(group_dir, f"{id_val}.parquet")
            if os.path.exists(path):
                group_df = pl.read_parquet(path)
                X, y = generate_sequences_for_id_polars(group_df, input_cols, input_len, output_len, stride)
                X_all.extend(X)
                y_all.extend(y)

        if X_all:
            np.save(os.path.join(save_dir, f"X_batch_{idx}.npy"), np.array(X_all, dtype=np.float32))
            np.save(os.path.join(save_dir, f"y_batch_{idx}.npy"), np.array(y_all, dtype=np.float32))

        # 🧹 Memory cleanup
        gc.collect()

    print(f"✅ Done saving all batches in {(time.time() - start) / 60:.2f} minutes")

In [29]:
# === Final Run ===
serial_generate_from_disk(
    GROUP_SAVE_DIR, 
    SAVE_DIR, 
    input_cols, 
    INPUT_SEQ_LEN, 
    FORECAST_HORIZON, 
    STRIDE, 
    BATCH_SIZE)

Saving shuffled batches: 100%|███████████████████████████████████████████████████████| 305/305 [42:24<00:00,  8.34s/it]

✅ Done saving all batches in 42.40 minutes


## Step 4.3 | Verify Training Sequences

### Step 4.2.1 | Load and Inspect File Shapes

In [15]:
# Example: Load the first batch
X = np.load("sequence_chunks_v3/X_batch_0.npy")
y = np.load("sequence_chunks_v3/y_batch_0.npy")

for i in range(5):
    print(f"Seq #{i} ➜ First input day d_num_scaled: {X[i, 0, -1]:.3f} → First target day sales: {y[i, 0]:.2f}")
    
print("✅ X shape:", X.shape)  # Expect: (# sequences, 56, # input features)
print("✅ y shape:", y.shape)  # Expect: (# sequences, 28)

Seq #0 ➜ First input day d_num_scaled: 0.000 → First target day sales: 0.00
Seq #1 ➜ First input day d_num_scaled: 0.004 → First target day sales: 0.00
Seq #2 ➜ First input day d_num_scaled: 0.007 → First target day sales: 0.00
Seq #3 ➜ First input day d_num_scaled: 0.011 → First target day sales: 0.00
Seq #4 ➜ First input day d_num_scaled: 0.014 → First target day sales: 0.00
✅ X shape: (26200, 56, 29)
✅ y shape: (26200, 28)


### Step 4.2.2 | Check Dimensional Consistency

In [9]:
assert X.shape[0] == y.shape[0], "❌ Mismatch: X and y have different number of sequences"
assert X.shape[1] == 56, "❌ Unexpected input sequence length"
assert y.shape[1] == 28, "❌ Unexpected forecast horizon length"
print("✅ Sequence dimensions are consistent")

✅ Sequence dimensions are consistent


### Step 4.2.3 | Sanity Check Input Ranges

Look for:
- Reasonable input values (not all 0s or NaNs)
- Normalized columns close to 0 mean / 1 std
- Categorical/flag columns like snap_CA are 0/1

In [18]:
print("✅ Sample input window:", X[0, :, :10])  # First 10 features of first sequence
print("✅ Sample target:", y[0])

✅ Sample input window: [[ 7.3100000e+02  2.0000000e+00  0.0000000e+00  6.0000000e+00
   1.0000000e+00  0.0000000e+00  1.9000000e+01  2.0000000e+00
  -9.7492790e-01 -2.2252093e-01]
 [ 7.3100000e+02  2.0000000e+00  0.0000000e+00  6.0000000e+00
   1.0000000e+00  0.0000000e+00  1.9000000e+01  2.0000000e+00
  -7.8183150e-01  6.2348980e-01]
 [ 7.3100000e+02  2.0000000e+00  0.0000000e+00  6.0000000e+00
   1.0000000e+00  0.0000000e+00  1.9000000e+01  2.0000000e+00
   0.0000000e+00  1.0000000e+00]
 [ 7.3100000e+02  2.0000000e+00  0.0000000e+00  6.0000000e+00
   1.0000000e+00  0.0000000e+00  1.9000000e+01  2.0000000e+00
   7.8183150e-01  6.2348980e-01]
 [ 7.3100000e+02  2.0000000e+00  0.0000000e+00  6.0000000e+00
   1.0000000e+00  0.0000000e+00  1.9000000e+01  2.0000000e+00
   9.7492790e-01 -2.2252093e-01]
 [ 7.3100000e+02  2.0000000e+00  0.0000000e+00  6.0000000e+00
   1.0000000e+00  0.0000000e+00  1.9000000e+01  2.0000000e+00
   4.3388373e-01 -9.0096885e-01]
 [ 7.3100000e+02  2.0000000e+00  0.

### Step 4.2.4 | Check Batch Distribution

In [11]:
sequence_counts = []
for i in range(305):
    X_batch = np.load(f"sequence_chunks_v3/X_batch_{i}.npy")
    sequence_counts.append(len(X_batch))

print(f"✅ Sequences per batch (mean): {np.mean(sequence_counts):.2f}")
print(f"ℹ️ Distribution: min={min(sequence_counts)}, max={max(sequence_counts)}")

✅ Sequences per batch (mean): 26191.41
ℹ️ Distribution: min=23580, max=26200


### Step 4.2.5 | Sequence Summary CSV

In [25]:
# === Configuration ===
SEQ_DIR = SAVE_DIR
GROUP_DIR = os.path.join(SEQ_DIR, "grouped_v3")
NUM_BATCHES = 305

In [26]:
# === Helper: Count sequences for each ID ===
def count_sequences_for_id(path, input_cols, input_len, output_len, stride):
    try:
        df = pl.read_parquet(path).sort("d_num")
        if df.height < input_len + output_len:
            return 0
        return (df.height - input_len - output_len) // stride + 1
    except Exception:
        return 0

In [28]:
# === Step 1: ID-level Summary ===
id_summary = []
parquet_files = sorted([f for f in os.listdir(GROUP_DIR) if f.endswith(".parquet")])

for f in parquet_files:
    id_val = f.replace(".parquet", "")
    path = os.path.join(GROUP_DIR, f)
    num_sequences = count_sequences_for_id(path, input_cols, INPUT_SEQ_LEN, FORECAST_HORIZON, STRIDE)
    id_summary.append({"id": id_val, "num_sequences": num_sequences})

df_id = pd.DataFrame(id_summary)
df_id.to_csv(os.path.join(SEQ_DIR, "id_sequence_summary.csv"), index=False)

In [29]:
# === Step 2: Batch-level Summary ===
batch_summary = []
for i in range(NUM_BATCHES):
    x_path = os.path.join(SEQ_DIR, f"X_batch_{i}.npy")
    y_path = os.path.join(SEQ_DIR, f"y_batch_{i}.npy")

    if os.path.exists(x_path) and os.path.exists(y_path):
        X = np.load(x_path)
        Y = np.load(y_path)
        batch_summary.append({
            "batch_index": i,
            "file_X": f"X_batch_{i}.npy",
            "file_Y": f"y_batch_{i}.npy",
            "num_sequences": len(X),
            "input_seq_len": X.shape[1] if X.ndim == 3 else "❌",
            "num_features": X.shape[2] if X.ndim == 3 else "❌",
            "forecast_horizon": Y.shape[1] if Y.ndim == 2 else "❌",
        })
    else:
        batch_summary.append({
            "batch_index": i,
            "file_X": f"X_batch_{i}.npy",
            "file_Y": f"y_batch_{i}.npy",
            "num_sequences": "Missing",
            "input_seq_len": "Missing",
            "num_features": "Missing",
            "forecast_horizon": "Missing",
        })

df_batch = pd.DataFrame(batch_summary)
df_batch.to_csv(os.path.join(SEQ_DIR, "batch_sequence_summary.csv"), index=False)

print("✅ CSV summaries saved!")

✅ CSV summaries saved!


## Step 4.4 | Create Validation Sequences

In [31]:
def generate_final_val_sequence_for_id_polars(group_df, input_cols, input_len=56, output_len=28):
    """
    Generates exactly one final validation sequence for a given id:
    Input: d_1858–1913 (56 days)
    Output: d_1914–1941 (28 days)
    """
    group_df = group_df.sort("d_num")
    
    # Find the window that ends input at d_1913
    last_input_end = 1913
    input_start = last_input_end - input_len + 1
    output_start = last_input_end + 1
    output_end = output_start + output_len - 1

    # Ensure the group has all required days
    if group_df.filter((pl.col("d_num") >= input_start) & (pl.col("d_num") <= output_end)).height < input_len + output_len:
        return [], []

    df_slice = group_df.filter((pl.col("d_num") >= input_start) & (pl.col("d_num") <= output_end))
    input_array = df_slice.filter(pl.col("d_num") <= last_input_end).select(input_cols).to_numpy().astype(np.float32)
    output_array = df_slice.filter(pl.col("d_num") >= output_start).select("sales").to_numpy().flatten().astype(np.float32)

    return [input_array], [output_array]  # wrap in list for consistent formatting

In [32]:
def serial_generate_val_sequences_from_disk(group_dir, save_dir, input_cols, input_len=56, output_len=28):
    """
    Loads each id’s parquet file from group_dir and saves the final val sequence
    for that id to save_dir as a single batch (all ids together).
    """
    os.makedirs(save_dir, exist_ok=True)

    X_all, y_all = [], []
    all_ids = [f.replace(".parquet", "") for f in os.listdir(group_dir) if f.endswith(".parquet")]

    start = time.time()
    for id_val in tqdm(all_ids, desc="Saving final val sequences"):
        path = os.path.join(group_dir, f"{id_val}.parquet")
        if os.path.exists(path):
            group_df = pl.read_parquet(path)
            X, y = generate_final_val_sequence_for_id_polars(group_df, input_cols, input_len, output_len)
            X_all.extend(X)
            y_all.extend(y)

    if X_all:
        np.save(os.path.join(save_dir, "X_val_final.npy"), np.array(X_all, dtype=np.float32))
        np.save(os.path.join(save_dir, "y_val_final.npy"), np.array(y_all, dtype=np.float32))

    print(f"✅ Done saving final validation sequences in {(time.time() - start) / 60:.2f} minutes")

In [ ]:
VAL_SAVE_DIR = "val_sequences_polars"

serial_generate_val_sequences_from_disk(
    GROUP_SAVE_DIR, 
    VAL_SAVE_DIR, 
    input_cols, 
    input_len=INPUT_SEQ_LEN, 
    output_len=FORECAST_HORIZON
)

## Step 4.5 | Verify Validation Sequences

In [33]:
# === Load the files ===
X_val = np.load("val_sequences_polars/X_val_final.npy")
y_val = np.load("val_sequences_polars/y_val_final.npy")

print(f"✅ Loaded X shape: {X_val.shape} — Expected: (num_ids, 56, num_features)")
print(f"✅ Loaded y shape: {y_val.shape} — Expected: (num_ids, 28)")

✅ Loaded X shape: (30490, 56, 29) — Expected: (num_ids, 56, num_features)
✅ Loaded y shape: (30490, 28) — Expected: (num_ids, 28)


In [34]:
# === Sanity Check 1: Number of days ===
assert X_val.shape[1] == 56, "Input window should be 56 days"
assert y_val.shape[1] == 28, "Output window should be 28 days"

In [35]:
# === Sanity Check 2: Examine one id’s sequence ===
idx = 0  # you can loop through more
x_sample = X_val[idx]
y_sample = y_val[idx]

print("\n📊 Sample input features for one id (shape):", x_sample.shape)
print("🧾 Sample output values:", y_sample)


📊 Sample input features for one id (shape): (56, 29)
🧾 Sample output values: [2. 0. 0. 0. 0. 1. 1. 0. 6. 1. 0. 3. 0. 0. 0. 2. 0. 0. 2. 3. 1. 0. 0. 0.
 1. 0. 0. 0.]


In [36]:
# === Optional: match against original parquet file ===
parquet_dir = "sequence_chunks_v3/grouped_v3"
sample_id = os.listdir(parquet_dir)[idx].replace(".parquet", "")
df = pl.read_parquet(os.path.join(parquet_dir, f"{sample_id}.parquet"))
df = df.sort("d_num")

In [37]:
# Filter the window that should match this X/Y
input_df = df.filter((pl.col("d_num") >= 1858) & (pl.col("d_num") <= 1913))
output_df = df.filter((pl.col("d_num") >= 1914) & (pl.col("d_num") <= 1941))

# === Extract raw arrays from .parquet
input_features_raw = input_df.select(input_cols).to_numpy().astype(np.float32)
output_sales_raw = output_df.select("sales").to_numpy().flatten().astype(np.float32)

In [38]:
# === Compare and validate
print(f"\n✅ Comparing sample_id: {sample_id}")
print("🔍 X_val match:", np.allclose(x_sample, input_features_raw, atol=1e-5))
print("🔍 y_val match:", np.allclose(y_sample, output_sales_raw, atol=1e-5))

# Optional: show diffs if mismatch
if not np.allclose(x_sample, input_features_raw, atol=1e-5):
    print("\n❌ Feature mismatch! Sample differences (X):")
    print(np.abs(x_sample - input_features_raw).max())

if not np.allclose(y_sample, output_sales_raw, atol=1e-5):
    print("\n❌ Sales mismatch! Sample differences (y):")
    print(np.abs(y_sample - output_sales_raw).max())


✅ Comparing sample_id: FOODS_1_001_CA_1_evaluation
🔍 X_val match: True
🔍 y_val match: True


## Step 4.6 | Create Evaluation Sequences

In [14]:
def generate_eval_input_sequence_for_id_polars(group_df, input_cols, input_start=1886, input_end=1941):
    """
    Extracts the final 56-day input window (d_1886–1941) for evaluation.
    Output is not generated — this is used for Kaggle submission prediction (d_1942–1969).
    """
    group_df = group_df.sort("d_num")

    df_slice = group_df.filter((pl.col("d_num") >= input_start) & (pl.col("d_num") <= input_end))
    
    if df_slice.height < (input_end - input_start + 1):
        return []

    input_array = df_slice.select(input_cols).to_numpy().astype(np.float32)
    return [input_array]  # wrapped in list for batch-style output

In [15]:
def serial_generate_eval_sequences_from_disk(group_dir, save_dir, input_cols, input_start=1886, input_end=1941):
    """
    Loads each id’s parquet file, extracts d_1886–1941 inputs,
    and saves:
        - X_eval_final.npy: 3D array of inputs
        - ids_eval_final.npy: list of id strings in matching order
    """
    os.makedirs(save_dir, exist_ok=True)

    X_all = []
    id_list = []
    all_ids = [f.replace(".parquet", "") for f in os.listdir(group_dir) if f.endswith(".parquet")]

    start = time.time()
    for id_val in tqdm(all_ids, desc="Saving evaluation input sequences"):
        path = os.path.join(group_dir, f"{id_val}.parquet")
        if os.path.exists(path):
            group_df = pl.read_parquet(path)
            X = generate_eval_input_sequence_for_id_polars(group_df, input_cols, input_start, input_end)
            if X:
                X_all.extend(X)
                id_list.append(id_val)

    if X_all:
        np.save(os.path.join(save_dir, "X_eval_final.npy"), np.array(X_all, dtype=np.float32))
        np.save(os.path.join(save_dir, "ids_eval_final.npy"), np.array(id_list))

    print(f"✅ Done saving evaluation inputs and ids in {(time.time() - start) / 60:.2f} minutes")

In [19]:
EVAL_SAVE_DIR = "eval_sequences_polars"

serial_generate_eval_sequences_from_disk(
    group_dir=GROUP_SAVE_DIR,
    save_dir=EVAL_SAVE_DIR,
    input_cols=input_cols,
    input_start=1886,
    input_end=1941
)

Saving evaluation input sequences: 100%|█████████████████████████████████████████| 30490/30490 [23:08<00:00, 21.95it/s]


✅ Done saving evaluation inputs and ids in 23.16 minutes


## Step 4.7 | Verify Evaluation Sequences

In [39]:
# === Load the files ===
X_eval = np.load("eval_sequences_polars/X_eval_final.npy")

print(f"✅ Loaded X shape: {X_eval.shape} — Expected: (num_ids, 56, num_features)")

✅ Loaded X shape: (30490, 56, 29) — Expected: (num_ids, 56, num_features)


In [40]:
# === Sanity Check 1: Number of days ===
assert X_eval.shape[1] == 56, "Input window should be 56 days"

In [41]:
# === Sanity Check 2: Examine one id’s sequence ===
idx = 0  # you can loop through more
x_sample = X_eval[idx]

print("\n📊 Sample input features for one id (shape):", x_sample.shape)


📊 Sample input features for one id (shape): (56, 29)


In [42]:
# === Optional: match against original parquet file ===
parquet_dir = "sequence_chunks_v3/grouped_v3"
sample_id = os.listdir(parquet_dir)[idx].replace(".parquet", "")
df = pl.read_parquet(os.path.join(parquet_dir, f"{sample_id}.parquet"))
df = df.sort("d_num")

In [43]:
# Filter the window that should match this X/Y
input_df = df.filter((pl.col("d_num") >= 1886) & (pl.col("d_num") <= 1941))

# === Extract raw arrays from .parquet
input_features_raw = input_df.select(input_cols).to_numpy().astype(np.float32)

In [44]:
# === Compare and validate
print(f"\n✅ Comparing sample_id: {sample_id}")
print("🔍 X_eval match:", np.allclose(x_sample, input_features_raw, atol=1e-5))

# Optional: show diffs if mismatch
if not np.allclose(x_sample, input_features_raw, atol=1e-5):
    print("\n❌ Feature mismatch! Sample differences (X):")
    print(np.abs(x_sample - input_features_raw).max())


✅ Comparing sample_id: FOODS_1_001_CA_1_evaluation
🔍 X_eval match: True


## Step 4.8 | Create Tail Training Sequences 

### Step 4.8.1 | Map IDs with Batch Numbers

In [48]:
def regenerate_and_save_batch_mapping(group_dir, batch_size=100, seed=42, output_csv="id_batch_map.csv"):
    all_ids = [f.replace(".parquet", "") for f in os.listdir(group_dir) if f.endswith(".parquet")]
    random.seed(seed)
    random.shuffle(all_ids)

    batch_map = []
    for batch_num, i in enumerate(range(0, len(all_ids), batch_size)):
        batch_ids = all_ids[i:i + batch_size]
        for id_val in batch_ids:
            batch_map.append({"id": id_val, "batch_num": batch_num})

    df = pd.DataFrame(batch_map)
    df.to_csv(output_csv, index=False)
    print(f"✅ Saved id–batch mapping to {output_csv}")

In [50]:
regenerate_and_save_batch_mapping(
    group_dir="sequence_chunks_v3/grouped_v3",
    batch_size=100,
    seed=42,
    output_csv="id_batch_mapping.csv"
)

✅ Saved id–batch mapping to id_batch_mapping.csv


### Step 4.8.2 | Generate Tail Training Sequences

In [51]:
def generate_tail_sequences_by_batch(group_dir, save_dir, input_cols, input_len=56, output_len=28, stride=7, seed=42, batch_size=100):
    """
    For each original training batch, generate final tail sequences (d_1914–1941)
    and save them per batch to maintain id alignment.
    """
    os.makedirs(save_dir, exist_ok=True)

    batch_id_groups = regenerate_batch_id_groups(group_dir, batch_size, seed)

    for idx, id_list in enumerate(tqdm(batch_id_groups, desc="Saving tail sequences by batch")):
        X_all, y_all = [], []
        for id_val in id_list:
            path = os.path.join(group_dir, f"{id_val}.parquet")
            if os.path.exists(path):
                group_df = pl.read_parquet(path).sort("d_num")
                df_len = group_df.height
                if df_len < input_len + output_len:
                    continue

                input_array = group_df.select(input_cols).to_numpy()
                sales_array = group_df.select("sales").to_numpy().flatten()

                for i in range(0, df_len - input_len - output_len + 1, stride):
                    target_window = group_df[i + input_len : i + input_len + output_len]
                    target_last_day = target_window[-1, "d_num"]
                    
                    # Only include sequences where prediction reaches d_1914+
                    if 1914 <= target_last_day <= 1941:
                        X_all.append(input_array[i:i + input_len].astype(np.float32))
                        y_all.append(sales_array[i + input_len:i + input_len + output_len].astype(np.float32))

        if X_all:
            np.save(os.path.join(save_dir, f"X_batch_{idx}_tail.npy"), np.array(X_all, dtype=np.float32))
            np.save(os.path.join(save_dir, f"y_batch_{idx}_tail.npy"), np.array(y_all, dtype=np.float32))

    print("✅ Tail sequence generation complete.")

In [52]:
FINAL_TAIL_SAVE_DIR = "final_train_tail_sequences"

generate_tail_sequences_by_batch(
    group_dir=GROUP_SAVE_DIR,
    save_dir=FINAL_TAIL_SAVE_DIR,
    input_cols=input_cols,
    input_len=56,
    output_len=28,
    stride=7,
    seed=42,
    batch_size=100
)

Saving tail sequences by batch: 100%|████████████████████████████████████████████████| 305/305 [21:16<00:00,  4.18s/it]

✅ Tail sequence generation complete.


In [53]:
X = np.load("final_train_tail_sequences/X_batch_0_tail.npy")
print("Shape:", X.shape)  # Should be (# sequences for batch 0 tail, 56, num_features)

Shape: (400, 56, 29)


# Step 5 | Create Final Training Sequences

## Step 5.1 | Setup

In [61]:
MAIN_DIR = "sequence_chunks_v3"
TAIL_DIR = "final_train_tail_sequences"
OUTPUT_DIR = "final_train_data"

os.makedirs(OUTPUT_DIR, exist_ok=True)

X_merged = []
y_merged = []

## Step 5.2 | Identify All Batch Indices in Main Dir

In [62]:
batch_numbers = sorted(set(
    int(f.split("_")[2].replace(".npy", ""))
    for f in os.listdir(MAIN_DIR)
    if f.startswith("X_batch_") and not "_tail" in f
))

print(f"🔢 Total batch indices found: {len(batch_numbers)}")
print("Preview:", batch_numbers[:5])

🔢 Total batch indices found: 305
Preview: [0, 1, 2, 3, 4]


## Step 5.3 | Load and Merge Main + Tail per Batch

In [63]:
gc.collect()

1700

In [64]:
for batch_num in tqdm(batch_numbers, desc="📦 Merging main + tail batches"):
    # Main paths
    x_main = os.path.join(MAIN_DIR, f"X_batch_{batch_num}.npy")
    y_main = os.path.join(MAIN_DIR, f"y_batch_{batch_num}.npy")

    X = np.load(x_main)
    y = np.load(y_main)

    # Tail paths
    x_tail = os.path.join(TAIL_DIR, f"X_batch_{batch_num}_tail.npy")
    y_tail = os.path.join(TAIL_DIR, f"y_batch_{batch_num}_tail.npy")

    if os.path.exists(x_tail) and os.path.exists(y_tail):
        X_tail = np.load(x_tail)
        y_tail = np.load(y_tail)

        X = np.concatenate([X, X_tail], axis=0)
        y = np.concatenate([y, y_tail], axis=0)

        print(f"🧩 Batch {batch_num:>3}: Main={X.shape[0] - X_tail.shape[0]:>4} | Tail={X_tail.shape[0]:>4} | Total={X.shape[0]:>4}")
    else:
        print(f"🧩 Batch {batch_num:>3}: Main={X.shape[0]:>4} | Tail=None | Total={X.shape[0]:>4}")

    np.save(os.path.join(OUTPUT_DIR, f"X_batch_{batch_num}.npy"), X)
    np.save(os.path.join(OUTPUT_DIR, f"y_batch_{batch_num}.npy"), y)

📦 Merging main + tail batches:   0%|                                                          | 0/305 [00:00<?, ?it/s]

🧩 Batch   0: Main=26200 | Tail= 400 | Total=26600


📦 Merging main + tail batches:   0%|▏                                                 | 1/305 [00:00<02:40,  1.89it/s]

🧩 Batch   1: Main=26200 | Tail= 400 | Total=26600


📦 Merging main + tail batches:   1%|▎                                                 | 2/305 [00:01<02:59,  1.69it/s]

...

📦 Merging main + tail batches:  99%|███████████████████████████████████████████████▌| 302/305 [03:46<00:02,  1.34it/s]

🧩 Batch 302: Main=26200 | Tail= 400 | Total=26600


📦 Merging main + tail batches: 100%|███████████████████████████████████████████████▊| 304/305 [03:47<00:00,  1.33it/s]

🧩 Batch 303: Main=26200 | Tail= 400 | Total=26600
🧩 Batch 304: Main=23580 | Tail= 360 | Total=23940


📦 Merging main + tail batches: 100%|████████████████████████████████████████████████| 305/305 [03:48<00:00,  1.33it/s]


In [65]:
# === Load one batch
BATCH_IDX = 0  # Try a few like 0, 100, 200, etc.

X = np.load(os.path.join(OUTPUT_DIR, f"X_batch_{BATCH_IDX}.npy"))
y = np.load(os.path.join(OUTPUT_DIR, f"y_batch_{BATCH_IDX}.npy"))

print(f"🧪 Batch {OUTPUT_DIR} shapes:")
print("X shape:", X.shape)  # Expect: (num_sequences, 56, 29)
print("y shape:", y.shape)  # Expect: (num_sequences, 28)

🧪 Batch final_train_data shapes:
X shape: (26600, 56, 29)
y shape: (26600, 28)


In [66]:
# Check last 3 y sequences — this is where tail sequences likely live
print("\n📈 Sample y values (last 3 rows):")
print(y[-3:])


📈 Sample y values (last 3 rows):
[[1. 1. 0. 3. 0. 3. 2. 0. 0. 0. 0. 3. 2. 0. 0. 2. 2. 0. 2. 0. 2. 1. 0. 0.
  1. 0. 0. 1.]
 [0. 0. 0. 0. 3. 2. 0. 0. 2. 2. 0. 2. 0. 2. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1.
  3. 0. 0. 2.]
 [0. 2. 2. 0. 2. 0. 2. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 3. 0. 0. 2. 0. 0. 1.
  0. 0. 0. 1.]]


In [71]:
# === Settings
BATCH_IDX = 0
ID_TO_CHECK = "FOODS_3_119_TX_3_evaluation"  # Pick an ID known to be in batch 0
GROUP_DIR = "sequence_chunks_v3\grouped_v3"
OUTPUT_DIR = "final_train_data"

In [72]:
# === Load original df for ID
df = pl.read_parquet(os.path.join(GROUP_DIR, f"{ID_TO_CHECK}.parquet")).sort("d_num")

# === Extract most recent 56 days (input window)
input_df = df.filter((pl.col("d_num") >= 1886) & (pl.col("d_num") <= 1941)).select("d_num", "sales")
print("🎯 Input window d_nums (last 3):", input_df.select("d_num").to_series()[-3:])

🎯 Input window d_nums (last 3): shape: (3,)
Series: 'd_num' [i16]
[
	1939
	1940
	1941
]


In [73]:
# === Load X/y arrays
X_batch = np.load(os.path.join(OUTPUT_DIR, f"X_batch_{BATCH_IDX}.npy"))
y_batch = np.load(os.path.join(OUTPUT_DIR, f"y_batch_{BATCH_IDX}.npy"))

# === Print a few final y values
print("\n🧾 Final y_batch rows (last 3):")
for row in y_batch[-3:]:
    print(row)

# === Shape confirmation
print("\n✅ X shape:", X_batch.shape)
print("✅ y shape:", y_batch.shape)


🧾 Final y_batch rows (last 3):
[1. 1. 0. 3. 0. 3. 2. 0. 0. 0. 0. 3. 2. 0. 0. 2. 2. 0. 2. 0. 2. 1. 0. 0.
 1. 0. 0. 1.]
[0. 0. 0. 0. 3. 2. 0. 0. 2. 2. 0. 2. 0. 2. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1.
 3. 0. 0. 2.]
[0. 2. 2. 0. 2. 0. 2. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 3. 0. 0. 2. 0. 0. 1.
 0. 0. 0. 1.]

✅ X shape: (26600, 56, 29)
✅ y shape: (26600, 28)
